In [1]:
from numpy import *
from PIL import *
import pickle
from pylab import *
import os

In [2]:
import sift
import dsift
dsift = reload(dsift)
import imtools
imtools = reload(imtools)

In [3]:
def read_gesture_features_labels(path):
    # make a list of the files with .dsift at the end
    featlist = [os.path.join(path, f) for f in os.listdir(path)
               if f.endswith('.dsift')]
    
    # read features
    features = []
    for featfile in featlist:
        l, d = sift.read_features_from_file(featfile)
        features.append(d.flatten())
    features = array(features)
    
    # generate labels
    labels = [featfile.split('/')[-1][0] for featfile in featlist]
    
    return features, array(labels)

In [4]:
features, labels = read_gesture_features_labels('train2/')
test_features, test_labels = read_gesture_features_labels('test2/')
classnames = unique(labels)

In [5]:
# the first letter of the file name is the label
print labels

['V' 'A' 'A' 'B' 'P' 'B' 'P' 'C' 'F' 'A' 'P' 'V' 'F' 'F' 'P' 'B' 'P' 'F'
 'F' 'B' 'P' 'F' 'F' 'A' 'F' 'P' 'A' 'A' 'F' 'V' 'A' 'P' 'V' 'V' 'P' 'A'
 'P' 'C' 'B' 'B' 'C' 'P' 'C' 'V' 'P' 'V' 'V' 'C' 'B' 'C' 'V' 'P' 'P' 'B'
 'B' 'C' 'A' 'C' 'V' 'B' 'V' 'B' 'F' 'P' 'A' 'P' 'B' 'P' 'B' 'V' 'F' 'B'
 'C' 'A' 'C' 'C' 'F' 'F' 'V' 'P' 'C' 'F' 'P' 'A' 'A' 'F' 'P' 'B' 'A' 'V'
 'F' 'V' 'C' 'C' 'B' 'C' 'A' 'F' 'F' 'A' 'V' 'C' 'V' 'V' 'A' 'B' 'A' 'C'
 'B' 'C' 'B' 'B' 'V' 'A' 'C' 'V' 'C' 'F' 'C' 'F' 'F' 'P' 'A' 'B' 'P' 'P'
 'B' 'B' 'A' 'F' 'B' 'A' 'C' 'C' 'V' 'A' 'C' 'P' 'P' 'V' 'F' 'V' 'P' 'A'
 'F' 'A' 'V' 'F' 'F' 'C' 'C' 'B' 'V' 'P' 'B' 'V' 'P' 'V' 'A' 'C' 'V' 'A'
 'V' 'A' 'F' 'F' 'B' 'A' 'B' 'F' 'C' 'B' 'C' 'P']


In [6]:
import knn

In [7]:
# k neighbors
k = 1
knn_classifier = knn.KnnClassifier(labels, features)

In [8]:
res = array([knn_classifier.classify(test_features[i], k) for i in
            range(len(test_labels))])

# accuracy
acc = sum(1.0*(res==test_labels))/len(test_labels)
print 'Accuracy:', acc

Accuracy: 0.242718446602


Accuracy is lower than the number in the text book (0.811518324607)

In [9]:
def print_confusion(res, test_labels, classnames):
    
    n = len(classnames)
    
    class_ind = dict([(classnames[i], i) for i in range(n)])
    
    confuse = zeros((n, n))
    for i in range(len(test_labels)):
        confuse[class_ind[res[i]], class_ind[test_labels[i]]] += 1
    
    print 'Confusion matrix for'
    print classnames
    print confuse

In [10]:
print_confusion(res, test_labels, classnames)

Confusion matrix for
['A' 'B' 'C' 'F' 'P' 'V']
[[  1.   0.   2.   2.   0.   0.]
 [  2.   5.   0.  10.   7.   5.]
 [  2.   0.   7.   1.   1.   0.]
 [  1.   3.   1.   6.   0.   0.]
 [  0.   0.   4.   0.   2.   0.]
 [  4.   4.   4.  10.  15.   4.]]


In [11]:
import pca

In [74]:
V, S, m = pca.pca(array(features))
V = V[:75]

In [75]:
features2 = [dot(V, f-m) for f in features]
test_features2 = [dot(V, f-m) for f in test_features]

In [76]:
knn_classifier2 = knn.KnnClassifier(labels, features2)

In [77]:
res2 = array([knn_classifier2.classify(test_features2[i], k) for i in
            range(len(test_labels))])

# accuracy
acc2 = sum(1.0*(res2==test_labels))/len(test_labels)
print 'Accuracy:', acc2

Accuracy: 0.271844660194


In [78]:
print_confusion(res2, test_labels, classnames)

Confusion matrix for
['A' 'B' 'C' 'F' 'P' 'V']
[[  1.   0.   1.   2.   0.   0.]
 [  2.   6.   0.  10.   6.   4.]
 [  2.   0.   8.   0.   0.   0.]
 [  1.   3.   1.   6.   0.   0.]
 [  0.   0.   4.   0.   2.   0.]
 [  4.   3.   4.  11.  17.   5.]]


Around 50-100 seems to be the sweet spot. Just before the curve in figure 8-9